In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
import requests

In [8]:
# import selenium to scrape the dyanmic website
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time


from selenium.webdriver.common.action_chains import ActionChains

from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()

C:\Users\CC\AppData\Local\Temp\ipykernel_9644\3873272524.py:19: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [9]:
#set url
url = 'https://okadabooks.com/store/best-sellers'
driver.get(url)
# keep clicking as long as there is something to click
while True:
    time.sleep(3)
    try:
        element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="best-sellers"]/div[2]/div/div[2]/a'))
        WebDriverWait(driver, 10).until(element_present)
        btn = driver.find_element(By.XPATH, '//*[@id="best-sellers"]/div[2]/div/div[2]/a')
        btn.click()
    except ElementClickInterceptedException:     
        ActionChains(driver).move_to_element(btn).click().perform()
    except TimeoutException:
        print("Timeout exception")
        break

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="best-sellers"]/div[2]/div/div[2]/a"}
  (Session info: chrome=104.0.5112.81)
Stacktrace:
Backtrace:
	Ordinal0 [0x011278B3+2193587]
	Ordinal0 [0x010C0681+1771137]
	Ordinal0 [0x00FD41A8+803240]
	Ordinal0 [0x010024A0+992416]
	Ordinal0 [0x0100273B+993083]
	Ordinal0 [0x0102F7C2+1177538]
	Ordinal0 [0x0101D7F4+1103860]
	Ordinal0 [0x0102DAE2+1170146]
	Ordinal0 [0x0101D5C6+1103302]
	Ordinal0 [0x00FF77E0+948192]
	Ordinal0 [0x00FF86E6+952038]
	GetHandleVerifier [0x013D0CB2+2738370]
	GetHandleVerifier [0x013C21B8+2678216]
	GetHandleVerifier [0x011B17AA+512954]
	GetHandleVerifier [0x011B0856+509030]
	Ordinal0 [0x010C743B+1799227]
	Ordinal0 [0x010CBB68+1817448]
	Ordinal0 [0x010CBC55+1817685]
	Ordinal0 [0x010D5230+1856048]
	BaseThreadInitThunk [0x77273744+36]
	RtlSetCurrentTransaction [0x778D9E54+212]
	RtlSetCurrentTransaction [0x778D9E1F+159]


In [10]:
#parse the source from driver to beautiful soup
source = driver.page_source
soup = bs(source,'lxml')

In [11]:
len(soup)

1

In [12]:
# get all the book links, prices and indexes
# create a function
links = []
listings = soup.find_all(class_ ="d-flex") # use beautifulsoup find_all
for listing in listings:
    bk_link = listing.find('a')['href']
    base= 'https://okadabooks.com'
    full_link = base + bk_link
    links.append(full_link) #append to empty lists

In [14]:
# loop for prices
p = []
prices = soup.find_all("div", class_="mini__book-card")
for price in prices:
    k = price.find('span', class_ ='price').getText()
    bk = {'prices':k}
    p.append(bk)

col = ['prices']
price=pd.DataFrame(p,columns=col)

In [15]:
#write price to csv
price.to_csv('price.csv',index=False, encoding='utf-8')

In [16]:
#loop for indexes
i = []
indexes = soup.find_all('div', class_="d-md-flex position-box")
for index in indexes:
    l = index.find('span').getText()
    inde = {'indexes':l}
    i.append(inde)

col = ['indexes']
indexess=pd.DataFrame(i,columns=col)

In [17]:
#write index to csv
indexess.to_csv('index.csv',index=False, encoding='utf-8')

In [18]:
#loop for link column still using the link cell
lint = []
listings = soup.find_all(class_ ="d-flex") # use beautifulsoup find_all
for listing in listings:
    bk_link = listing.find('a')['href']
    base= 'https://okadabooks.com'
    full_link = base + bk_link
    lint.append(full_link) #append to empty lists
    
col = ['site']
sites=pd.DataFrame(lint,columns=col)

In [19]:
#write links to csv
sites.to_csv('links.csv',index=False, encoding='utf-8')

In [20]:
# check length of set
print(len(links))
print(len(price))
print(len(indexess))
print(len(sites))

2480
2480
2480
2480


In [23]:
#create an empty list to store all books
okada_books = []
# set column list
col = ['title','author','ratings','download_count']
# create a for loop to iterate through the links and get key columns
for link in links:
    req = requests.get(link).text
    book_soup = bs(req, 'html.parser')
    title = book_soup.find('title').text
    author = book_soup.find('strong').text
    ratings = book_soup.find('div', class_='engagement-stats d-flex justify-content-between').find('span').text.strip()
    download_count = book_soup.find('div', class_='engagement-stats d-flex justify-content-between').find(title="Download Count").text.strip()
    #print(title,author,ratings,download_count)
    #break
    
    book = {'title':title, 'author':author, 'ratings':ratings, 'download_count':download_count}
    okada_books.append(book) #append to empty list

# write to pandas dataframe
df = pd.DataFrame(okada_books, columns=col)

In [24]:
# write to csv file
df.to_csv('okada_bk.csv', index =False, encoding='Utf-8')